In [ ]:
!pip install tensorflow ultralytics

In [2]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import Adam
from glob import glob
import matplotlib.pyplot as plt
import os, shutil, random
from tensorflow.keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings("ignore", message="Transparent hugepages are not enabled")
from ultralytics import YOLO
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model = YOLO('yolov8n.pt')

In [5]:
print(model.model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

In [6]:
import os
import yaml

#Modify this to your dataset root path
dataset_root = "/content/drive/MyDrive/object_detection"

#Define paths
train_images = os.path.join(dataset_root, "train/images")
val_images = os.path.join(dataset_root, "valid/images")

data_yaml = {
    'train': train_images,
    'val': val_images,
    'nc': 6,  # number of classes
    'names': ['Clean', 'Dusty', 'Bird-Drop', 'Electrical-Damage', 'Physical-Damage', 'Snow-Covered']
}

# Path to save YAML file
yaml_path = os.path.join(dataset_root, 'solarpanel_data.yaml')

# Save YAML file
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"YAML file created successfully at: {yaml_path}")


YAML file created successfully at: /content/drive/MyDrive/object_detection/solarpanel_data.yaml


In [10]:
!yolo detect train data=/content/drive/MyDrive/object_detection/data.yaml model=yolov8n.pt epochs=50 imgsz=640

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/object_detection/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0,

In [8]:
model.train(
    data="/content/drive/MyDrive/object_detection/data.yaml",
    epochs=15,
    imgsz=640,
    batch=5,
    workers=8,
    device = 0,        # set -1 for CPU
    name="solar_defect_yolov8n"
)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=5, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/object_detection/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=solar_defect_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d04807245c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [11]:
model = YOLO("/content/runs/detect/solar_defect_yolov8n/weights/best.pt")

results = model.predict(
    source="/content/drive/MyDrive/object_detection/test/images/7_JPG.rf.ce375f060388f4e0d7ef007ff493ab1d.jpg",
    save=True,
    imgsz=640,
    conf=0.1
)

# img_path = "/content/drive/MyDrive/Solarpanel/test_image.jpg"
# results = model(img_path, save=True)

print(results)




image 1/1 /content/drive/MyDrive/object_detection/test/images/7_JPG.rf.ce375f060388f4e0d7ef007ff493ab1d.jpg: 640x640 2 Bird Drops, 8.0ms
Speed: 3.8ms preprocess, 8.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/detect/predict
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Bird Drop', 1: 'Clean', 2: 'Dusty', 3: 'Electrical Damage', 4: 'Physical Damage', 5: 'Snow Covered'}
obb: None
orig_img: array([[[142, 137, 139],
        [124, 119, 121],
        [124, 119, 121],
        ...,
        [236, 248, 248],
        [244, 255, 255],
        [244, 255, 255]],

       [[127, 122, 124],
        [113, 108, 110],
        [122, 117, 119],
        ...,
        [233, 245, 245],
        [234, 246, 246],
        [234, 246, 246]],

       [[112, 109, 111],
        [105, 102, 104],
        [125, 122, 124],
        ...,
        [226, 236, 236],
        

In [ ]:
results = model.predict(
    source="/content/drive/MyDrive/object_detection/train/images",
    save=True,
    conf=0.05
)

print(results)

In [ ]:
metrics = model.val()
print(metrics)